In [ ]:
# Plotting for chatbot arena experiments

import inverse_cai.paper_plotting
import pathlib

import importlib
importlib.reload(inverse_cai.paper_plotting)

USER1_PATH = pathlib.Path("../exp/outputs/saved_outputs/anthropic-hh-v3/harmless")
USER2_PATH = pathlib.Path("../exp/outputs/saved_outputs/anthropic-hh-v3/helpful")

user1_train = inverse_cai.paper_plotting.get_metrics_dict(USER1_PATH, dataset="testset_0")
user1_test = inverse_cai.paper_plotting.get_metrics_dict(USER1_PATH, dataset="testset_1")
user2_train = inverse_cai.paper_plotting.get_metrics_dict(USER2_PATH, dataset="testset_1")
user2_test = inverse_cai.paper_plotting.get_metrics_dict(USER2_PATH, dataset="testset_0")

best_user1_constitution = user1_train["gpt4omini_fn_constitutional_base_neutral_v2"]["max_constitution"]#["constitutions"][1]
best_user2_constitution = user2_train["gpt4omini_fn_constitutional_base_neutral_v2"]["max_constitution"]#["constitutions"][2]

appendix_dir = pathlib.Path("./appendix/constitutions/prism_cross_group")
annotator = "gpt4omini_fn_constitutional_base_neutral_v2"
inverse_cai.paper_plotting.write_constitutions_to_file(user1_train[annotator], appendix_dir / "user1")
inverse_cai.paper_plotting.write_constitutions_to_file(user2_train[annotator], appendix_dir / "user2")


combined_data = {
    "Harmless annotations": {
        "": {
            "Harmless const.": user1_train["gpt4omini_fn_constitutional_base_neutral_v2"],
            "Default": user1_train["alpaca_eval_gpt4omini_fn_noinstruction"],
            "Helpful const.": user2_test["gpt4omini_fn_constitutional_base_neutral_v2"],
        },
    },
    "Helpful annotations": {
        "": {
            "Harmless const.": user1_test["gpt4omini_fn_constitutional_base_neutral_v2"],
            "Default": user2_train["alpaca_eval_gpt4omini_fn_noinstruction"],
            "Helpful const.": user2_train["gpt4omini_fn_constitutional_base_neutral_v2"],
        },
        }
}

inverse_cai.paper_plotting.plot_data(
    combined_data,
    "figures/0050_experiments_helpful_harmless.png",
    colors= ["#f5cbcc",  "#d9ead3", "#fff3cc"],
    hatches= ["\\\\","//", ""],
    add_constitutions=True,
    legend_loc="lower right",
    constitutions=[best_user1_constitution, best_user2_constitution],
    constitution_colors=["#f5cbcc", "#fff3cc"],
)

# We want to add the model name to the table, even if it is the same everywhere
combined_data_with_model_names = {setting: {"GPT-4o-mini": data_dict[""]} for setting, data_dict in combined_data.items()}
num_seeds = len(combined_data["Harmless annotations"][""]["Harmless const."]["values"])
latex_table = inverse_cai.paper_plotting.generate_latex_table(combined_data_with_model_names, f"Results for cross-group experiments on PRISM data. Averaged over {num_seeds}~random seeds.", "tab:prism_cross_group_num_ersults", "appendix/numerical_results/anthropic_hh_cross_group.tex")
print(latex_table)